In [ ]:
!pip install gradio bypy huggingface_hub

In [ ]:
import os
import shutil

from huggingface_hub import snapshot_download

from bypy import ByPy

### 下载模型

In [ ]:
def copy_data(source_directory,target_directory):
  # 遍历源目录中的所有文件和目录
  for entry in os.listdir(source_directory):
      entry_path = os.path.join(source_directory, entry)
      # 检查是否为符号链接
      if os.path.islink(entry_path):
          # 获取符号链接的目标路径
          target_path = os.readlink(entry_path)
          # 构建目标路径的绝对路径
          target_absolute_path = os.path.join(source_directory, target_path)
          # 获取目标路径的文件名或目录名
          target_basename = os.path.basename(target_absolute_path)
          # 构建目标文件或目录的路径
          target_entry_path = os.path.join(target_directory, entry)

          # 复制目标文件或目录到目标目录
          if os.path.isdir(target_absolute_path):
              shutil.copytree(target_absolute_path, target_entry_path)
          else:
              shutil.copy2(target_absolute_path, target_entry_path)
      else:
          # 如果不是符号链接，直接复制到目标目录
          shutil.copy2(entry_path, os.path.join(target_directory, entry))

In [ ]:
def download(repo_id):
    source_directory = snapshot_download(repo_id=repo_id,cache_dir="/app/data/",force_download =True)
    print("文件路径:",source_directory)
    path_name = repo_id.replace("/","---")
    target_directory = f"{os.getcwd()}/download/{path_name}"
    # 移动文件
    os.system(f"mkdir -p {target_directory}")
    os.system(f"mkdir -p {os.getcwd()}/tar/{path_name}/")
    copy_data(source_directory,target_directory)
    print("复制路径：",target_directory)

    return target_directory

In [ ]:
## 设置 repo_id
repo_id = "THUDM/chatglm2-6b-int4"

In [ ]:
out_path = download(repo_id)

In [ ]:
out_path

### 上传百度网盘

百度云开发者平台： https://pan.baidu.com/union

大小限制

所有开发者均可接入使用接口，但可上传单个文件大小根据授权用户的身份有不同的限制:

- 普通用户单个上传文件大小上限为4GB
- 会员用户单个上传文件大小上限为10GB
- 超级会员用户单个上传文件大小上限为20GB

In [ ]:
def upload_by_file(source_path):
    bp = ByPy()
    dest_path = source_path.split("/")[-1]  # 文件名
    out = bp.upload(
      source_path,
      dest_path
    )
    return out
def upload_by_path(source_path,compress=True):
    bp = ByPy()
    # 创建目录
    dest_path = source_path.split("/")[-1]
    # 压缩
    if compress:
      print("分卷压缩....")
      tar_path =f"{os.getcwd()}/tar/{dest_path}"
      os.system(f"zip -s 4000M {tar_path}/{dest_path}.zip {source_path}/*")
      source_path = tar_path
    print("上传百度云....")
    bp.mkdir(dest_path)
    # 同步目录
    out = bp.syncup(
      source_path,
      dest_path
    )
    return out

In [ ]:
bp = ByPy()
print(bp.list())

In [ ]:
# 上传文件夹
# 如果单个文件大小小于 4G 则无需压缩
upload_by_path(out_path,compress=False)

### 删除模型

In [ ]:
!rm -rf /content/download/*
!rm -rf /content/tar/*
